In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


Dataset create from coco json


In [9]:
import os
import json
import math
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm

# -------------------
# CONFIG
# -------------------


ANNOTATIONS_JSON = r"/content/drive/MyDrive/labels_test_catarct.json"   # <- your COCO JSON from makesense.ai
IMAGES_ROOT      = r"/content/drive/MyDrive/processed_images/test/cataract"        # <- folder containing images (can be nested)
OUTPUT_DIR       = r"/content/drive/MyDrive/processed_images_pupil/test/cataract"                      # <- where to save cropped images

PADDING          = 0.1                           # % padding around bbox


def clamp(val, lo, hi):
    return max(lo, min(val, hi))


def main():
    with open(ANNOTATIONS_JSON, "r") as f:
        coco = json.load(f)

    images_by_id = {img["id"]: img for img in coco["images"]}
    out_root = Path(OUTPUT_DIR)
    out_root.mkdir(parents=True, exist_ok=True)

    saved, skipped = 0, 0

    for ann in tqdm(coco["annotations"], desc="Cropping circular"):
        img_id = ann["image_id"]
        img_info = images_by_id.get(img_id)
        if not img_info:
            skipped += 1
            continue

        img_path = Path(IMAGES_ROOT) / img_info["file_name"]
        if not img_path.exists():
            skipped += 1
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            skipped += 1
            continue
        H, W = img.shape[:2]

        if "bbox" not in ann:
            skipped += 1
            continue

        x, y, w, h = ann["bbox"]

        # Add padding
        px, py = PADDING * w, PADDING * h
        x1 = clamp(int(x - px), 0, W)
        y1 = clamp(int(y - py), 0, H)
        x2 = clamp(int(x + w + px), 0, W)
        y2 = clamp(int(y + h + py), 0, H)

        # Crop region
        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            skipped += 1
            continue

        # Create circular mask
        h_crop, w_crop = crop.shape[:2]
        cx, cy = w_crop // 2, h_crop // 2
        radius = min(cx, cy)
        mask = np.zeros((h_crop, w_crop), dtype=np.uint8)
        cv2.circle(mask, (cx, cy), radius, 255, -1)

        # Apply mask with black background
        result = np.zeros_like(crop)  # black background
        for c in range(3):
            result[:,:,c] = cv2.bitwise_and(crop[:,:,c], crop[:,:,c], mask=mask)

        # Save as JPG (3-channel, no alpha)
        base = Path(img_info["file_name"]).stem
        out_name = f"{base}_img{img_id}_ann{ann['id']}.jpg"
        out_path = out_root / out_name
        cv2.imwrite(str(out_path), result)
        saved += 1

    print(f"\n✅ Done. Saved: {saved} circular crops, Skipped: {skipped}.")


if __name__ == "__main__":
    main()


Cropping circular: 100%|██████████| 61/61 [00:03<00:00, 15.80it/s]


✅ Done. Saved: 61 circular crops, Skipped: 0.
